A database with the boroughs' boundaries was downloaded, and this code is an attempt to locate each crime within the boundaries of the boroughs, and generate the borough name in that way. 

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
import requests
import json
import pandas as pd
from pprint import pprint as pp

boroughs = pd.read_csv('/content/gdrive/MyDrive/Group_7_Files/data/data/csv/airbnb-111222-140323/neighbourhoods.csv')
crime_data = pd.read_csv('/content/gdrive/MyDrive/Group_7_Files/data/data/csv/police_uk_data_london_metropolitan/2023-03/2023-03-metropolitan-street.csv')

# convert easting and northing boundaries into long and lat
# for this API easting and northing must be integers
def converter_long(easting, northing):
    easting = round(easting)
    northing = round(northing)
    i = f'https://api.getthedata.com/bng2latlong/{easting}/{northing}'
    response = requests.get(url=i)
    data = response.json()
    long = data['longitude']
    return long

def converter_lat(easting, northing):
    easting = round(easting)
    northing = round(northing)
    i = f'https://api.getthedata.com/bng2latlong/{easting}/{northing}'
    response = requests.get(url=i)
    data = response.json()
    lat = data['latitude']
    return lat


# convert to longitude and latitude
boroughs['lat'] = boroughs[['x', 'y']].apply(lambda row: converter_lat(row['x'], row['y']), axis=1)
boroughs['long'] = boroughs[['x', 'y']].apply(lambda row: converter_long(row['x'], row['y']), axis=1)

# establish latitude and longitude boundaries
lat_max = boroughs.groupby('borough')['lat'].max()
lat_min = boroughs.groupby('borough')['lat'].min()
long_max = boroughs.groupby('borough')['long'].max()
long_min = boroughs.groupby('borough')['long'].min()

# check which borough lat, long of crime data falls into to create new column
def check_boundary(lat, long):
    for x, row in boroughs.iterrows():
        if row['lat_min'] <= lat <= row['lat_max'] and row['long_min'] <= long <= row['long_max']:
            return row['borough']
    return 'isna'

# test the code on a small extract
crime_data2 = crime_data.head(5)

# create a new column with the borough names from the boundary
crime_data2['borough'] = crime_data2[['Latitude', 'Longitude']].apply(lambda row: check_boundary(row['Latitude'], row['Longitude']), axis=1)

crime_data2['borough']

The final lamdaa function did not fully work, and there were still issues with space-time complexity. At the time of  working on improving, another team member developed the data cleaning solution, and so this code was abandoned. 
